# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [4]:
df = data.groupby(['CustomerID', 'ProductName']).agg({'Quantity':'sum'})
df.head()

Quantity
CustomerID ProductName                            
33         Apricots - Dried                      1
           Assorted Desserts                     1
           Bandage - Flexible Neon               1
           Bar Mix - Pina Colada, 355 Ml         1
           Beans - Kidney, Canned                1

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [5]:
prodCustom = pd.pivot_table(df, values='Quantity', index='ProductName', columns='CustomerID', fill_value=0, dropna=False)
prodCustom

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yeast Dry - Fermipan,0,0,0,0,0,0,0,0,0,0,...,0,0,0,25,0,0,0,0,0,0
Yoghurt Tubes,0,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,25,0,0
"Yogurt - Blueberry, 175 Gr",0,1,0,0,0,0,0,0,0,0,...,25,0,0,25,0,0,0,0,0,0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [13]:
dist_matrix = pdist(prodCustom.T, metric='euclidean')
dist_matrix

array([ 11.91637529,  10.48808848,  11.22497216, ..., 304.13812651,
       305.16389039, 303.10889132])

In [14]:
customSim = squareform(dist_matrix)
customSim

array([[  0.        ,  11.91637529,  10.48808848, ..., 228.62851966,
        239.        , 229.77380181],
       [ 11.91637529,   0.        ,  11.74734012, ..., 228.01096465,
        239.03765394, 229.70415756],
       [ 10.48808848,  11.74734012,   0.        , ..., 228.08112592,
        238.26665734, 229.77380181],
       ...,
       [228.62851966, 228.01096465, 228.08112592, ...,   0.        ,
        304.13812651, 305.16389039],
       [239.        , 239.03765394, 238.26665734, ..., 304.13812651,
          0.        , 303.10889132],
       [229.77380181, 229.70415756, 229.77380181, ..., 305.16389039,
        303.10889132,   0.        ]])

## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [102]:
similar = pd.DataFrame(1/(1 + customSim), 
                         index=prodCustom.columns, columns=prodCustom.columns)
mostSim = similar[33][1:].sort_values(ascending=False).nlargest(5)
list(mostSim.index)

[3535, 264, 3317, 2503, 3305]

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [56]:
records = df.loc[mostSim.index]
records

Quantity
CustomerID ProductName                             
264        Apricots - Halves                      1
           Apricots Fresh                         1
           Bacardi Breezer - Tropical             1
           Bagel - Plain                          1
           Banana - Leaves                        1
...                                             ...
3535       Tray - 16in Rnd Blk                    1
           Wanton Wrap                            1
           Wine - Blue Nun Qualitatswein          1
           Wine - Magnotta, Merlot Sr Vqa         1
           Yoghurt Tubes                          1

[270 rows x 1 columns]

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [75]:
records_1 = records.groupby('ProductName').agg({'Quantity':'sum'}).sort_values('Quantity', ascending=False)
records_1

,Quantity
ProductName,
Butter - Unsalted,3
Wine - Ej Gallo Sierra Valley,3
Towels - Paper / Kraft,3
Soup - Campbells Bean Medley,3
Wine - Blue Nun Qualitatswein,3
...,...
Hinge W Undercut,1
Ice Cream Bar - Hageen Daz To,1
Jagermeister,1


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [89]:
prodCustom[[33]].merge(records_1, left_on='ProductName', right_on='ProductName').sort_values('Quantity', ascending=False).nlargest(5, 'Quantity')[prodCustom[33] == 0]

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,33,Quantity
ProductName,,
Butter - Unsalted,0,3
Wine - Ej Gallo Sierra Valley,0,3
Soup - Campbells Bean Medley,0,3
Wine - Blue Nun Qualitatswein,0,3


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [141]:
customers = list(set(data.CustomerID))
recom = {}
for customer in customers[:5]:
    similar= pd.DataFrame(1/(1 + squareform(pdist(prodCustom.T, 'euclidean'))), 
                         index=prodCustom.columns, columns=prodCustom.columns)
    mostSim = similar[customer][1:].sort_values(ascending=False).nlargest(5).index.tolist()
    records = df.loc[mostSim]
    records_1 = records.groupby('ProductName').agg({'Quantity':'sum'}).sort_values('Quantity', ascending=False)
    customerN = prodCustom[[customer]]
    sales = customerN.merge(records_1, left_on='ProductName', right_on='ProductName')[prodCustom[customer] == 0].sort_values('Quantity', ascending=False).nlargest(5, 'Quantity')
    recom[f'{customer}']=sales.index.tolist()
    
recom

# I print only 5 first customers

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # Remove the CWD from sys.path while we load stuff.


{'83973': ['Cocoa Butter',
  'Veal - Slab Bacon',
  'Bandage - Fexible 1x3',
  'Ezy Change Mophandle',
  'Milk Powder'],
 '59399': ['Shrimp - Baby, Warm Water',
  'Nantucket - Pomegranate Pear',
  'Towels - Paper / Kraft',
  'Snapple - Iced Tea Peach',
  'Bagel - Plain'],
 '92168': ['Cup - Translucent 7 Oz Clear',
  'Veal - Brisket, Provimi,bnls',
  'Snapple Lemon Tea',
  'Spinach - Baby',
  'Pop Shoppe Cream Soda'],
 '49159': ['Pasta - Penne, Rigate, Dry',
  'Cookies - Assorted',
  'Cheese - Parmesan Grated',
  'Zucchini - Yellow',
  'Rice - Jasmine Sented'],
 '18441': ['Soupfoamcont12oz 112con',
  'Sausage - Breakfast',
  'Flour - Pastry',
  'Foam Cup 6 Oz',
  'Soup Knorr Chili With Beans']}

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [165]:
recomdf = pd.DataFrame.from_dict(recom)

reco = recomdf.T
reco.columns = ('1', '2', '3', '4', '5')
reco.index.name = 'CustomerID'
reco

,1,2,3,4,5
CustomerID,,,,,
83973,Cocoa Butter,Veal - Slab Bacon,Bandage - Fexible 1x3,Ezy Change Mophandle,Milk Powder
59399,"Shrimp - Baby, Warm Water",Nantucket - Pomegranate Pear,Towels - Paper / Kraft,Snapple - Iced Tea Peach,Bagel - Plain
92168,Cup - Translucent 7 Oz Clear,"Veal - Brisket, Provimi,bnls",Snapple Lemon Tea,Spinach - Baby,Pop Shoppe Cream Soda
49159,"Pasta - Penne, Rigate, Dry",Cookies - Assorted,Cheese - Parmesan Grated,Zucchini - Yellow,Rice - Jasmine Sented
18441,Soupfoamcont12oz 112con,Sausage - Breakfast,Flour - Pastry,Foam Cup 6 Oz,Soup Knorr Chili With Beans


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [171]:
#cosine

recom_cosine = {}
for customer in customers[:5]:
    similar= pd.DataFrame(1/(1 + squareform(pdist(prodCustom.T, 'cosine'))), 
                         index=prodCustom.columns, columns=prodCustom.columns)
    mostSim = similar[customer][1:].sort_values(ascending=False).nlargest(5).index.tolist()
    records = df.loc[mostSim]
    records_1 = records.groupby('ProductName').agg({'Quantity':'sum'}).sort_values('Quantity', ascending=False)
    customerN = prodCustom[[customer]]
    sales = customerN.merge(records_1, left_on='ProductName', right_on='ProductName')[prodCustom[customer] == 0].sort_values('Quantity', ascending=False).nlargest(5, 'Quantity')
    recom_cosine[f'{customer}']=sales.index.tolist()
    
recom_cosineDF = pd.DataFrame.from_dict(recom_cosine).T
recom_cosineDF.columns = ['1', '2', '3', '4', '5']
recom_cosineDF.index.name = 'CustomerID'
recom_cosineDF

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # This is added back by InteractiveShellApp.init_path()


,1,2,3,4,5
CustomerID,,,,,
83973,"Cheese - Brie,danish",Pastry - Choclate Baked,Puree - Passion Fruit,Lettuce - Spring Mix,Broom - Corn
59399,Anchovy Paste - 56 G Tube,Ezy Change Mophandle,Garlic - Peeled,Broom - Corn,Cake - Mini Cheesecake
92168,"Beer - Alexander Kieths, Pale Ale",Nantucket - Pomegranate Pear,Tia Maria,"Wine - Red, Colio Cabernet",Pork - Inside
49159,Beer - Blue,Peas - Frozen,Lamb - Ground,Beef - Ground Medium,"Turnip - White, Organic"
18441,V8 - Berry Blend,Whmis - Spray Bottle Trigger,Tea - Jasmin Green,"Pasta - Penne, Rigate, Dry",Beef - Inside Round
